In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob # forgot what for?...
import numpy as np

In [ ]:
scp_items = pd.read_csv(r'.\data_evolutionary_game_theory\items_scp_evolgath_202302021205.csv', index_col=None)
scp_abstracts = pd.read_csv(r'.\data_evolutionary_game_theory\abstracts_scp_evolgath_202302021207.csv', index_col=None)
scp_refs = pd.read_csv(r'.\data_evolutionary_game_theory\refs_scp_evolgath_202302021207.csv', index_col=None)
wos_items = pd.read_csv(r'.\data_evolutionary_game_theory\items_wos_evolgath_202301271505.csv', index_col=None)
wos_abstracts = pd.read_csv(r'.\data_evolutionary_game_theory\abstracts_wos_evolgath_202301271503.csv', index_col=None)
wos_refs = pd.read_csv(r'.\data_evolutionary_game_theory\refs_wos_evolgath_202301271505.csv', index_col=None)

Start first steps for perspective networks:

In [ ]:
scp_items.loc[:,'item_title'] #only title column
wos_items.loc[:,'item_title'] #only title column
scp_items['item_title'].equals(wos_items['item_title']) #scp and wos do not hold the same publications

In [ ]:
# Which titles are included in both scp and wos?
list(set(scp_items.item_title) & set(wos_items.item_title)) 
#common values, Check whether scp is included within wos? or vice versa

In [ ]:
# Which item_id do certain titles hold?
scp_items.loc[scp_items['item_title'] == 'The N-Player Trust Game and its Replicator Dynamics', 'item_id']
wos_items.loc[wos_items['item_title'] == 'The N-Player Trust Game and its Replicator Dynamics', 'item_id'] 
#item_id format is different for wos and scp

# For later: How to handle scp and wos differences? For now, wos apparently holds more titles

Filter for duplicates:

In [ ]:
wos_items.item_id.nunique() # 1501
wos_refs.item_id_cited.nunique() #unique IDs for cited articles # 38708
#does this mean that the number of relevant work for the field is not too big?

In [ ]:
wos_refs['edited_title'] = wos_refs.ref_item_title.str.lower() #create new column with all titles in lower-case
wos_refs.loc[:,'edited_title'] #check whether lower-casing has worked; yes! Length: 59935
wos_refs.ref_item_title.nunique() #how many different titles would there be unedited? # 29667
wos_refs.edited_title.nunique() #how many different titles left when edited to lower-case?
# Length: only 29443!! (nunique drops NA by default, if not wanted, add dropna=False)
wos_refs['edited_title'].isna().sum() # 8435
wos_refs.edited_title.nunique(dropna=False) # NA counts as 1 # 29444
series = wos_refs.groupby("item_id_cited")['item_id_citing'].nunique() #group the cited id by the citing id!
print(series) # same output as for wos_refs.item_id_cited.nunique(), makes sense

In [ ]:
#plt.hist(series.values, bins=100);

#plt.xlim(0, 20)

In [ ]:
series.loc[series.values > 1] #cited more than once # 5291
wos_refs.loc[wos_refs.item_id_cited.isin(series.loc[series.values > 1].index)] 
#get where item_id_cited is in the series of cited more often than once

#do the following after we know about duplicates # EDIT: DONT!
#wos_refs[['item_id_cited', 'ref_pubyear', 'ref_authors']].drop_duplicates() 

In [ ]:
dictedtit={} #create an empty dictionary
for index,s in wos_refs.iterrows(): #loop over rows in our df
    if s['edited_title'] in dictedtit: #if title already exists in our created dictionary
        if s['item_id_cited'] not in dictedtit[s['edited_title']]: #and id_item_cited is still not present in our dictionary
            dictedtit[s['edited_title']].append(s['item_id_cited']) #then append the id_item_cited to the existing title
    else: 
        dictedtit[s['edited_title']] = [s['item_id_cited']] #if not, then create new key title with item_id_cited as a value
        
len(dictedtit) # 29444
print(dictedtit['toward a metabolic theory of ecology']) # testwise, gives item id cited for this title ['WOS:000223113500001']
# print(dictedtit) # IOPub data rate exceeded

print(list(dictedtit.items())[:100])

In [ ]:
newdict = {k: dictedtit[k] for k in list(dictedtit)[:20]}
# trying out how to get only some entries of the dictionary
print(newdict) #gives chunk of 20 titles


dictduplicate = {k: dictedtit[k] for k in dictedtit if len(dictedtit[k]) > 1} 
# only key-value pairs with >1 id_item_cited, because others should be fine and no need to relabel
print(dictduplicate)
len(dictduplicate) # 549

In [ ]:
wos_refs.isna()
print( sorted(dictduplicate.items(), key= lambda x: len(x[1]), reverse=True) )
#sort by number of values ()

del dictduplicate[np.nan]# nan needs the np in front!!
dictduplicatesorted = sorted(dictduplicate.items(), key= lambda x: len(x[1]), reverse=True) 
# sort by number of values
print(dictduplicatesorted)

In [ ]:
dfduplicated =pd.DataFrame.from_dict(dict(dictduplicatesorted), orient='index') # df for better visibility
dfduplicated.to_csv('titles_with_multi_ids.csv', encoding='utf-8')
dfduplicated #check only really for the bigger first one, the other ones are most probably the same

Create replacing dictionary and a new column for the wos_refs df

In [ ]:
wos_refs['item_id_clear'] = wos_refs['item_id_cited'] #clone item_id_cited column to later replace values only in there
wos_refs.head() #worked, new column created
type(dictduplicate) # dict

dictreplace ={v[0]: v[1:] for v in dictduplicate.values()} 
# ignore keys aka titles; take first id as key, remaining ids until last as values
# goal: {'BCI:BCI19522600023673': ["A", "B"]} 
dfdictreplace=pd.DataFrame.from_dict(dict(dictduplicatesorted), orient='index') # df for better visibility
dfdictreplace
# would need double looping through df and value lists, better try rearranging dictionary to long format

dictreplace_alt = {i: k for k,v in dictreplace.items() for i in v} # needs .items() wtf, otherwise returns meta stuff
# k becomes first id
# v is remaining list of other ids

# same as:
#dictreplace_alt = {}
#for k,v in dictreplace.items():
#    for i in v:
#        dictreplace_alt[i]=k

dictreplace_alt

In [ ]:
wos_refs2 = wos_refs.replace({'item_id_clear':dictreplace_alt})
wos_refs.compare(wos_refs2)

# check titles with most ids
wos_refs2.loc[wos_refs['edited_title'] == 'evolutionary game theory'] # title needs manual check
wos_refs2.loc[wos_refs['edited_title'] == 'stochastic evolutionary game dynamics'] # title needs manual check
wos_refs2.loc[wos_refs['edited_title'] == 'population dynamics from game theory'] # all the same
wos_refs2.loc[wos_refs['edited_title'] == 'evolutionary game theory: a renaissance'] # all the same
wos_refs2.loc[wos_refs['edited_title'] == 'random processes in genetics'] # all the same
wos_refs2.loc[wos_refs['edited_title'] == 'the emergence of commitments and cooperation'] # all the same, checked manually, deviation due to changed author order

wos_items.item_title.nunique()
wos_refs2.item_id_clear.nunique() # 37870

In [ ]:
## ATTENTION FOR FUTURE VIVI: INDEXING HAPPENING HERE!

# manual check for distinct articles with title "evolutionary game theory"
weibull = [2809,19703,21738,34994,48101,56850] # weibull
wos_refs2.loc[weibull,'item_id_clear'] = '000527769700042.48' 
wos_refs2.at[7354, 'item_id_clear'] = '000498881800052.4' # easley ; this guy's name is not easeley btw, can't find publication
sandholm = [8738, 9257, 19512, 52016, 53875] # sandholm
wos_refs2.loc[sandholm,'item_id_clear'] = '000296007000001.31'
alexander = [12024, 29023, 32574, 37342, 44444, 57100] # J. McKenzie Alexander
wos_refs2.loc[alexander,'item_id_clear'] = '000297819200015.9'
wos_refs2.at[27274, 'item_id_clear'] = '000607329000002.10' # cressman
wos_refs2.at[37946, 'item_id_clear'] = '000525401500057.17' # smith
wos_refs2.at[46560, 'item_id_clear'] = '000457952400133.21' # siegmund
wos_refs2.at[50085, 'item_id_clear'] = '000441998400054.33' # vincent


# manual check for distinct articles with title "stochastic evolutionary game dynamics"
pd.set_option('display.max_rows', None) # force jupyter notebook to show me all the rows
# need to reduce down from 68 rows
stoch = wos_refs2.loc[wos_refs2['edited_title'] == 'stochastic evolutionary game dynamics']  # title needs manual check
stoch.loc[stoch['item_id_cited'] != 'WOS:A1990EE29600005']
# foster authors can keep their given item_id_clear
wallace = [9259, 56675] # wallace
wos_refs2.loc[wallace,'item_id_clear'] = '000550684000036.11' 

traulsen = [9911, 10864, 11060, 13370, 16064, 18399, 21053, 
            21296, 26125, 30109, 34608, 36098, 43663, 47440, 48532, 50225, 58148] # traulsen & hauert 
wos_refs2.loc[traulsen,'item_id_clear'] = '000705615300001.39' 

In [ ]:
# drop useless columns that contain mostly nans
wos_refs2 = wos_refs2.drop(columns=['scopus_ref_issue', 
                        'wos_ref_article_number', 
                        'scopus_ref_text', 
                        'scopus_ref_fulltext', 
                        'wos_citation_context' ])
wos_refs2.head() # looks fine

In [ ]:
wos_refs2.to_csv('wos_refs_cleaned.csv')

1: break down and clean notebook!s! 2: clean items table and refs table and export clean csv; columns with mostly NaN can be deleted 3: zaggl paper: replicate section data set description and write it up; look up Gmür 2003 CoCit-value